In [ ]:
# Imports

import gensim
from gensim import corpora
from gensim import models, similarities
from gensim.similarities import Similarity
from gensim.models import LsiModel
import os
import collections
import smart_open
import random

import pandas as pd
import numpy as np


from pprint import pprint
import string

In [ ]:
!ls

In [ ]:
! mkdir wiki-archive
! mv *.txt wiki-archive/

In [ ]:
!ls

In [ ]:
# Paths

DATA_FOLDER = 'output_data'
DICTIONARY_PATH = 'products_corpus.dict'
MODEL_FILENAME = 'model'
SIMILARITIES_FILENAME = 'similarities'

In [ ]:
def load_dictionary():
    """
    loads dictionary from the disk
    """
    dictionary = corpora.Dictionary.load(os.path.join(DATA_FOLDER, DICTIONARY_PATH))
    
    return dictionary

In [ ]:
def load_model():
    """
    read model from the disk
    """
    
    model = LsiModel.load(MODEL_FILENAME)
    return model

In [ ]:
def load_similarities():
    """
    read similarity indexes from list
    """
    
    index = Similarity.load(SIMILARITIES_FILENAME)
    return index

### Load from disk

In [ ]:
model = load_model()
dictionary = load_dictionary()
index = load_similarities()

### Test Sample doc

In [ ]:
sample_doc = 'healthy green tea'
# Pre - process document
vec_bow = dictionary.doc2bow(sample_doc.lower().split())
vec_lsi = model[vec_bow]
print(vec_lsi)

In [ ]:
similarity_scores = index[vec_lsi]
similarity_scores = list(enumerate(similarity_scores))

# Sort in descending order of similarity
sorted_similarity_scores = sorted(similarity_scores, key=lambda tup: tup[1], reverse=True)

In [ ]:
# Reading product data from disk
def get_train_data():
    """
    SQL here 
    """
    data = pd.read_csv('~/Desktop/grocery_products.csv')
    return data

data = get_train_data()
selected_data = data[['source_product_id', 'name', 'departments_name', 'categories_name', 'description_en']]
selected_data['document'] = selected_data['name'] + ' ' + selected_data['departments_name'] + ' ' + selected_data['categories_name'] + ' ' + selected_data['description_en']

docs = selected_data['document'].tolist()
product_ids = selected_data['source_product_id'].tolist()

# Mapping from document index 
doc_index_to_id = {}
for index, product_id in enumerate(product_ids):
    doc_index_to_id[index] = product_id

In [ ]:
for doc_index, score in sorted_similarity_scores[:10]:
    product_id = doc_index_to_id[doc_index]
    product_info = str(selected_data[selected_data['source_product_id'] == product_id]['name'])
#     product_info += " : " + str(selected_data[selected_data['source_product_id'] == product_id]['description_en'])
    print(f'{product_info} : {score}')
#     print(str(selected_data[selected_data['source_product_id'] == product_id]['description_en']))